# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ciudad lazaro cardenas,17.9583,-102.2000,28.65,62,94,4.44,MX,1733693642
1,1,howrah,22.5892,88.3103,18.00,72,20,1.54,IN,1733693644
2,2,ellsworth,44.5434,-68.4195,1.59,92,100,2.06,US,1733693645
3,3,port-aux-francais,-49.3500,70.2167,2.23,98,100,12.93,TF,1733693646
4,4,oum hadjer,13.2954,19.6966,25.57,21,24,6.10,TD,1733693647


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Create the map plot using hvplot
map_plot = city_data_df.hvplot.points(
    x='Lng', y='Lat',
    size='Humidity', 
    color='Humidity',
    cmap='viridis',
    alpha=0.7, 
    title='City Weather Map',    
    xlabel='Longitude',           
    ylabel='Latitude',                                
    hover_cols=['City', 'Humidity'],
)

# Display the plot
map_plot

:Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] > 21) &
    (city_data_df['Max Temp'] < 27) & 
    (city_data_df['Wind Speed'] < 4.5) & 
    (city_data_df['Cloudiness'] == 0)
].dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
162,162,ziguinchor,12.5833,-16.2719,22.11,88,0,3.09,SN,1733693857
213,213,wailua homesteads,22.0669,-159.3780,26.18,79,0,3.60,US,1733693925
343,343,kapa'a,22.0752,-159.3190,26.93,79,0,3.60,US,1733694097
395,395,holualoa,19.6228,-155.9522,26.57,74,0,4.12,US,1733694170
428,428,artesia,33.8658,-118.0831,21.98,60,0,3.60,US,1733694215


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
162,ziguinchor,SN,12.5833,-16.2719,88,
213,wailua homesteads,US,22.0669,-159.3780,79,
343,kapa'a,US,22.0752,-159.3190,79,
395,holualoa,US,19.6228,-155.9522,74,
428,artesia,US,33.8658,-118.0831,60,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation.hotel',
    'apiKey': geoapify_key,
    'limit': 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ziguinchor - nearest hotel: Hotel Ndary
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
kapa'a - nearest hotel: Pono Kai Resort
holualoa - nearest hotel: Kona Hotel
artesia - nearest hotel: Days Inn & Suites by Wyndham Artesia


,City,Country,Lat,Lng,Humidity,Hotel Name
162,ziguinchor,SN,12.5833,-16.2719,88,Hotel Ndary
213,wailua homesteads,US,22.0669,-159.3780,79,Hilton Garden Inn Kauai Wailua Bay
343,kapa'a,US,22.0752,-159.3190,79,Pono Kai Resort
395,holualoa,US,19.6228,-155.9522,74,Kona Hotel
428,artesia,US,33.8658,-118.0831,60,Days Inn & Suites by Wyndham Artesia


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    x='Lng', y='Lat',
    size='Humidity',
    color='Humidity',
    cmap='viridis',
    alpha=0.7,
    title='Cities with Hotels - Humidity Representation',
    xlabel='Longitude',
    ylabel='Latitude',
    hover_cols=['City', 'Country', 'Hotel Name'],
    geo=True,
    tiles='OSM',
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)